In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df: pd.DataFrame = pd.read_csv('cac40.csv')
df.dtypes

In [ ]:
df["Close"].plot()
plt.show()

In [ ]:
diff = np.diff(df["Close"])
fig, ax = plt.subplots(figsize=(20,5))
df["Close"].plot(x="Date", ax=ax)
events = np.argwhere(diff > 150).ravel().astype(float)
plt.vlines(events, 0, df["Close"].max(), color="red", zorder=10)
plt.show()

In [ ]:
from tick.hawkes import HawkesExpKern

In [ ]:
betas = np.logspace(-4, 0, 100)
scores = []
for beta in betas:
    process = HawkesExpKern(beta).fit([events])
    scores.append(process.score([events]))
plt.semilogx(betas, scores)
plt.show()
best_score = np.argmax(scores)
print(f"Best beta: {betas[best_score]}")
process = HawkesExpKern(betas[best_score]).fit([events])
process.fit([events])
print(f"Best mu and alpha/beta: {process.coeffs[0], process.coeffs[1]}")
print(process.score([events]))
intens, x_steps = process.estimated_intensity([events], 1)
print(intens[0].shape, x_steps.shape)
fig, ax = plt.subplots(figsize=(20,5))
plt.plot(x_steps, intens[0])
plt.vlines(events, 0, intens[0].max(), color="red", alpha=.1, zorder=10)
plt.show()